In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [3]:
from KuhnPoker import KuhnPokerModel, KuhnPokerInfoSet, ActionNode, PASS, ADD_CHIP, Card, TensorModel, SamplingNode
from ISMCTS import Tree, Constants
from AlphaZero import NNModel, SelfPlayDataV, SelfPlayDataP

import numpy as np
import pickle
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch

In [4]:
start_ix = 0
num_model = 256

In [5]:
vmodel = []
pmodel = []

for i in range(start_ix, start_ix + num_model):
    vmodel.append(torch.load(f'model/vmodel-{i}.pt'))
    pmodel.append(torch.load(f'model/pmodel-{i}.pt'))

In [6]:
p_info_set = KuhnPokerInfoSet([0], [None, Card.JACK])
q_info_set = KuhnPokerInfoSet([0, 1], [Card.QUEEN, None])
j01_info_set = KuhnPokerInfoSet([0, 1], [None, Card.JACK])
kj01_info_set = KuhnPokerInfoSet([0, 1], [Card.KING, Card.JACK])
qj01_info_set = KuhnPokerInfoSet([0, 1], [Card.QUEEN, Card.JACK])

In [7]:
ps = []
qs = []
vs = []
vs1 = []
vs2 = []
for i in range(num_model):
    p = TensorModel(vmodel[i], pmodel[i]).eval_P(ActionNode(p_info_set))[1]
    q = TensorModel(vmodel[i], pmodel[i]).eval_P(ActionNode(q_info_set))[1]
    v, _ = TensorModel(vmodel[i], pmodel[i]).eval_V(SamplingNode(j01_info_set))
    kj_node = ActionNode(kj01_info_set)
    kj_node.spawned_tree = 1
    v1, _ = TensorModel(vmodel[i], pmodel[i]).eval_V(kj_node)
    qj_node = ActionNode(qj01_info_set)
    qj_node.spawned_tree = 1
    v2, _ = TensorModel(vmodel[i], pmodel[i]).eval_V(qj_node)
    ps.append(p)
    qs.append(q)
    vs.append(v)
    vs1.append(v1)
    vs2.append(v2)
df = pd.DataFrame({'p': ps, 'q': qs, 'v_?J[01]': vs, 'v_KJ[01]': vs1, 'v_QJ[01]': vs2})

In [8]:
df

,p,q,v_?J[01],v_KJ[01],v_QJ[01]
0,0.333333,0.333333,0.253393,0.031782,-0.085364
1,0.333333,0.333333,1.068813,0.685151,0.465528
2,0.333333,0.333333,1.074450,0.029165,0.074021
3,0.333333,0.333333,0.608339,-0.488714,-0.262401
4,0.333333,0.333333,0.510397,-0.825797,-0.411283
...,...,...,...,...,...
251,0.333333,0.333333,-0.768873,-1.939717,0.036598
252,0.333333,0.333333,-1.375398,-1.926805,0.039856
253,0.333333,0.333333,-0.840167,-2.021745,-0.087574
254,0.333333,0.333333,-0.879993,-2.116560,-0.270557


In [ ]:
plt.plot(df.index, df['v_?J[01]'])

In [ ]:
data = df

plt.figure(figsize=(8, 6)) 
plt.plot(data['p'], data['q'], marker='o', linestyle='-', color='b')
plt.title('Trajectory in p-q Space') 
plt.xlabel('p')  
plt.ylabel('q') 
plt.grid(True) 

# Annotate every 10th point with its order number
for i, (p, q) in enumerate(zip(data['p'], data['q'])):
    if i % 16 == 0: 
        plt.text(p, q, str(i), color="red", fontsize=12)
        
plt.scatter([1/3], [1/3], color='green', s=100, marker='x', label='Target (1/3, 1/3)')

plt.show()

In [ ]:
data = df

plt.figure(figsize=(8, 6)) 
plt.plot(data['p'], data['q'], marker='o', linestyle='-', color='b')
plt.title('Trajectory in p-q Space') 
plt.xlabel('p')  
plt.ylabel('q') 
plt.grid(True) 

# Annotate every 10th point with its order number
for i, (p, q) in enumerate(zip(data['p'], data['q'])):
    if i % 16 == 0: 
        plt.text(p, q, str(i), color="red", fontsize=12)
        
plt.scatter([1/3], [1/3], color='green', s=100, marker='x', label='Target (1/3, 1/3)')

plt.show()

In [ ]:
with open('self_play_games/self_play_games.pkl', 'rb') as f:
    games = pickle.load(f)

In [ ]:
games

In [ ]:
data = SelfPlayDataV(games)

In [ ]:
subset = []
for g in games:
    if g.info_set.action_history == [0, 1] and g.info_set.cards ==[Card.QUEEN, None]:
        subset.append(g)
df = pd.DataFrame(subset)
df['prob_1'] = df['action_dist'].apply(lambda x: x[1])

In [ ]:
pd.DataFrame(data)

In [ ]:
df['prob_1'].describe()

In [ ]:
df['outcome'].value_counts()

In [ ]:
policy_data = pd.DataFrame(games)

In [ ]:
policy_data